In [1]:
!pip install pandas xlsxwriter


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [126]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("records").getOrCreate()

ConnectionRefusedError: [Errno 111] Connection refused

In [124]:
spark.sql("DROP TABLE IF EXISTS demo.nyc.taxis_10M_50COLUMNS_where")

ConnectionRefusedError: [Errno 111] Connection refused

In [107]:
import os
iceberg_table_dir = "../warehouse/nyc/taxis_10M_50COLUMNS_where"
metadata_dir = f"{iceberg_table_dir}/metadata"
data_dir = f"{iceberg_table_dir}/data"
input_data_dir = f"../input_data"
analysis_info = []
records_before_op = 0

def append_to_file(file_path, msg):
    open_mode = "a"
    if not os.path.exists(file_path):
        open_mode = "w"

    # Open the CSV file in write mode
    with open(file_path, open_mode) as file:
        writer = csv.writer(file)
        
        if open_mode=="w":
            #writing header of the columns
            writer.writerows([list(msg.keys())])    

        row_values = [list(msg.values())]
        # Write the data to the CSV file
        writer.writerows(row_values)

def get_size():
    # List the metadata files
    manifest_pattern = re.compile(r".*-m\d+\.avro$")
    metadata_files = os.listdir(metadata_dir)
    
    # Initialize variables to store the sizes of different types of metadata files
    snap_avro_size = 0
    metadata_json_size = 0
    m_avro_size = 0

    data_dir_size = 0
    # get data dir size
    data_dir_files = os.listdir(data_dir)
    # print(data_dir_files)
    for filename in data_dir_files:
        file_path = os.path.join(data_dir, filename)
        data_dir_size += os.path.getsize(file_path) / 1024  # Convert size to KB
    
    # Iterate through the metadata files and calculate their sizes
    for file in metadata_files:
        file_path = os.path.join(metadata_dir, file)
        file_size_kb = os.path.getsize(file_path) / 1024  # Convert size to KB
        
        if file.startswith("snap-") and file.endswith(".avro"):
            snap_avro_size += file_size_kb
        elif file.endswith(".metadata.json"):
            metadata_json_size += file_size_kb
        elif manifest_pattern.match(file):
            m_avro_size += file_size_kb
    
    # Print the time taken and the sizes of the metadata files
    # print(f"Time taken to read Parquet files: {time_taken:.2f} seconds")
    # print(f"Size of snap-*.avro files: {snap_avro_size:.2f} KB")
    # print(f"Size of *.metadata.json files: {metadata_json_size:.2f} KB")
    # print(f"Size of *m{0-9}{1,}.avro files: {m_avro_size:.2f} KB")

    return {"data_dir_size": data_dir_size,"metadata_size": metadata_json_size,"snapshot_size": snap_avro_size,"manifest_size": m_avro_size}


In [108]:
from pyspark.sql.types import (
    DoubleType, FloatType, LongType, StructType, StructField, 
    StringType, IntegerType, DateType
)

# Define the schema with 50 columns based on the required data types
schema = StructType([
    # StructField("vendor_id", LongType(), True),  # INT
    # StructField("trip_id", LongType(), True),  # INT
    # StructField("trip_distance", FloatType(), True),  # FLOAT
    # StructField("fare_amount", DoubleType(), True),  # DOUBLE
    # StructField("store_and_fwd_flag", StringType(), True)  # STRING
# ] + [
    # Assigning VARCHAR, INT, STRING, and DATE data types in a cyclic pattern
    StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 0 else  # VARCHAR
    StructField(f"extra_col_{i}", IntegerType(), True) if i % 4 == 1 else  # INT
    StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 2 else  # STRING
    StructField(f"extra_col_{i}", DateType(), True)  # DATE
    for i in range(50)
])

# Create an empty DataFrame with the schema
df = spark.createDataFrame([], schema)

# Create the Iceberg table
df.writeTo("demo.nyc.taxis_10M_50COLUMNS_where").create()


In [109]:
df = spark.table("demo.nyc.taxis_10M_50COLUMNS_where")
df.show()

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|extra_col_0|extra_col_1|extra_col_2|extra_col_3|extra_col_4|extra_col_5|extra_col_6|extra_col_7|extra_col_8|extra_col_9|extra_col_10|extra_col_11|extra_col_12|extra_col_13|extra_col_14|extra_col_15|extra_col_16|extra_col_17|extra_col_18|extra_col_19|extra_col_20|extra_col_21|extra_col_22|extra_col_23|extra_col_24|extra_col_25|extra_col_26|extra_col_27|ext

In [105]:
import time, csv
from pyspark.sql.functions import col, when, lit
from pyspark.sql import functions as F
import os

input_data_dir = "../input_data"
output_dir = "../output"
analysis_info = []
records_before_op = 0
total_insertion_time = 0

file_type = input("Enter input file type csv or parquet? : ").lower().strip()
input_data_dir = os.path.join(input_data_dir, file_type)
input_files = os.listdir(input_data_dir)

analysis_file = os.path.join(output_dir, f"analysis_info_{file_type}.csv")
if os.path.exists(analysis_file):
    os.remove(analysis_file)

df = spark.table("demo.nyc.taxis_1M_50COLUMNS_where")
records_before_op = df.count()

for file in input_files:
    print(f"Started with file={file}")
    file_path = os.path.join(input_data_dir, file)

    st = time.time()
    if file_type == "parquet":
        df = spark.read.parquet(file_path)
    else:
        df = spark.read.csv(file_path, header=True)
        df = df.select(
            F.col("extra_col_0").cast("long"),
            F.col("extra_col_1").cast("int"),
            F.col("extra_col_2").cast("string"),
            F.col("extra_col_3").cast("date"),
            *[F.col(f"extra_col_{i}").cast("string" if i % 4 == 0 or i % 4 == 2 else "int" if i % 4 == 1 else "date") for i in range(4, 45)]
        )

    rows = df.count()
    
    df.writeTo("demo.nyc.taxis_1M_50COLUMNS_where").append()
    end = time.time() - st
    total_insertion_time += end

    details = {"time_taken": f"{end:.2f} sec", "Operation": f"Inserted {rows} records", "records_after_op": records_before_op + rows}
    records_before_op += rows

    print(f"Inserted {rows} records in {end:.2f} sec.")

# **PRINT INSERTION TIME BEFORE UPDATE**
print(f"\nTotal insertion time: {total_insertion_time:.2f} sec\n")

# === Get Update Percentage from User ===
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100

df = spark.table("demo.nyc.taxis_1M_50COLUMNS_where")
total_rows = df.count()

num_rows = int(total_rows * update_percentage)
print(f"Updating {num_rows} rows (~{update_percentage*100}%)...")

# Sample random rows ensuring unique IDs
sampled_df = df.select("extra_col_0").distinct().orderBy(F.rand()).limit(num_rows)
sampled_ids = [row["extra_col_0"] for row in sampled_df.collect()]

st = time.time()

# === Using WHERE Clause in Update ===
# Now we use `where` to filter out the rows that need to be updated
# The WHERE clause equivalent here will filter the rows we want to update (i.e., matching extra_col_0 values)
updated_df = df.withColumn(
    "extra_col_1",
    when(col("extra_col_0").isin(sampled_ids), col("extra_col_1") + 10)
    .otherwise(col("extra_col_1"))
)

# Overwrite the table partitions with the final updated data
updated_df.writeTo("demo.nyc.taxis_1M_50COLUMNS_where").overwritePartitions()

end = time.time() - st

print(f"Updated {num_rows} rows in {end:.2f} sec")


Enter input file type csv or parquet? :  parquet


Started with file=records_1000000_part_1_1740397925.6984496.parquet


Inserted 1000000 records in 36.04 sec.

Total insertion time: 36.04 sec



Enter update percentage (e.g., 1 for 1%):  10


Updating 100000 rows (~10.0%)...


25/02/26 20:06:54 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB


Updated 100000 rows in 161.63 sec


In [110]:
import time, csv
from pyspark.sql.functions import col, when, lit, broadcast
from pyspark.sql import functions as F
import os

input_data_dir = "../input_data"
output_dir = "../output"
analysis_info = []
records_before_op = 0
total_insertion_time = 0

file_type = input("Enter input file type csv or parquet? : ").lower().strip()
input_data_dir = os.path.join(input_data_dir, file_type)
input_files = os.listdir(input_data_dir)

analysis_file = os.path.join(output_dir, f"analysis_info_{file_type}.csv")
if os.path.exists(analysis_file):
    os.remove(analysis_file)

df = spark.table("demo.nyc.taxis_10M_50COLUMNS_where")
records_before_op = df.count()

for file in input_files:
    print(f"Started with file={file}")
    file_path = os.path.join(input_data_dir, file)

    st = time.time()
    if file_type == "parquet":
        df = spark.read.parquet(file_path)
    else:
        df = spark.read.csv(file_path, header=True)
        df = df.select(
            F.col("extra_col_0").cast("long"),
            F.col("extra_col_1").cast("int"),
            F.col("extra_col_2").cast("string"),
            F.col("extra_col_3").cast("date"),
            *[F.col(f"extra_col_{i}").cast("string" if i % 4 == 0 or i % 4 == 2 else "int" if i % 4 == 1 else "date") for i in range(4, 45)]
        )

    rows = df.count()
    
    df.writeTo("demo.nyc.taxis_10M_50COLUMNS_where").append()
    end = time.time() - st
    total_insertion_time += end

    details = {"time_taken": f"{end:.2f} sec", "Operation": f"Inserted {rows} records", "records_after_op": records_before_op + rows}
    records_before_op += rows

    print(f"Inserted {rows} records in {end:.2f} sec.")

# **PRINT INSERTION TIME BEFORE UPDATE**
print(f"\nTotal insertion time: {total_insertion_time:.2f} sec\n")

# === Get Update Percentage from User ===
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100

df = spark.table("demo.nyc.taxis_10M_50COLUMNS_where")
total_rows = df.count()

num_rows = int(total_rows * update_percentage)
print(f"Updating {num_rows} rows (~{update_percentage*100}%)...")

# Sample random rows ensuring unique IDs (this will be done for each batch)
sampled_df = df.select("extra_col_0").distinct().orderBy(F.rand()).limit(num_rows)
sampled_ids = [row["extra_col_0"] for row in sampled_df.collect()]

# Convert sampled_ids to a DataFrame to use with join
sampled_ids_df = spark.createDataFrame([(id,) for id in sampled_ids], ["extra_col_0"])

# Broadcast the sampled DataFrame to avoid broadcasting large task binary
broadcast_sampled_ids = broadcast(sampled_ids_df)

st = time.time()

# === Using JOIN for Update Operation ===
# Instead of using `withColumn()` and `isin()` for the update, we join the sampled_ids to the original dataframe
updated_df = df.join(broadcast_sampled_ids, on="extra_col_0", how="left_outer") \
    .withColumn(
        "extra_col_1",
        when(col("extra_col_0").isNotNull(), col("extra_col_1") + 10)
        .otherwise(col("extra_col_1"))
    ) \
    .drop("extra_col_0")  # Drop the extra_col_0 after the join since it's no longer needed

# Overwrite the table partitions with the final updated data
updated_df.writeTo("demo.nyc.taxis_10M_50COLUMNS_where").overwritePartitions()

end = time.time() - st

print(f"Updated {num_rows} rows in {end:.2f} sec")


Enter input file type csv or parquet? :  parquet


Started with file=records_1000000_part_10_1740401457.66906.parquet


Inserted 1000000 records in 49.76 sec.
Started with file=records_1000000_part_1_1740398687.6853974.parquet


Inserted 1000000 records in 60.01 sec.
Started with file=records_1000000_part_2_1740398997.7710938.parquet


Inserted 1000000 records in 55.86 sec.
Started with file=records_1000000_part_3_1740399303.6597402.parquet


Inserted 1000000 records in 57.21 sec.
Started with file=records_1000000_part_4_1740399611.4401598.parquet


Inserted 1000000 records in 56.89 sec.
Started with file=records_1000000_part_5_1740399918.8825066.parquet


25/02/26 20:18:18 ERROR Utils: Aborting task=========>              (3 + 1) / 4]
java.lang.OutOfMemoryError: Java heap space
	at org.apache.parquet.bytes.BytesInput$StreamBytesInput.toByteArray(BytesInput.java:285)
	at org.apache.parquet.bytes.BytesInput.toByteBuffer(BytesInput.java:237)
	at org.apache.parquet.bytes.BytesInput.toInputStream(BytesInput.java:246)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader.readPageV1(VectorizedColumnReader.java:367)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader.access$000(VectorizedColumnReader.java:49)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader$1.visit(VectorizedColumnReader.java:272)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader$1.visit(VectorizedColumnReader.java:268)
	at org.apache.parquet.column.page.DataPageV1.accept(DataPageV1.java:120)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader.rea

Py4JJavaError: An error occurred while calling o179024.append.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 375.0 failed 1 times, most recent failure: Lost task 1.0 in stage 375.0 (TID 488) (82e05dee74ee executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.parquet.bytes.BytesInput$StreamBytesInput.toByteArray(BytesInput.java:285)
	at org.apache.parquet.bytes.BytesInput.toByteBuffer(BytesInput.java:237)
	at org.apache.parquet.bytes.BytesInput.toInputStream(BytesInput.java:246)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader.readPageV1(VectorizedColumnReader.java:367)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader.access$000(VectorizedColumnReader.java:49)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader$1.visit(VectorizedColumnReader.java:272)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedColumnReader$1.visit(Vectori